# Assignment 3
### Name: Jason Hellwig
### CWID: 894789627

## Code take from assignment specificiation

In [1]:
import numpy as np


class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad


class MultiplyNode(object):
    """
    Multiplies two inputs
    """

    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output

    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient


class AddNode(object):
    """
    Adds two inputs x1 and x2.
    """

    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output

    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient


class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """

    def forward(self, x):
        self.x = x
        self.output = Node(1 / (1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output

    def backward(self):
        s = 1 / (1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

## Modified perceptron class
### Generalized for an arbitrary amount of imput weights

In [2]:
class Perceptron(object):
    def __init__(self, inputs, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha

        ### Initializing weights/bias to a random float between -1 and 1.
        self.w = [Node(np.random.uniform(-1, 1), 0.0) for x in inputs]
        self.w.append(Node(np.random.uniform(-1, 1), 0.0))  # bias term

        ### Input and Output variables
        self.x = [Node(x, 0.0) for x in inputs]
        self.x.append(Node(1, 0.0))  # bias term

        ### Initialize operators nodes required
        ### for processing the inputs within a perceptron
        self.initialize_operators()

    def initialize_operators(self):
        self.mul = [MultiplyNode() for node in self.w]
        self.add = [AddNode() for node in self.w[:-1]]  # there is n-1 add operators
        self.sigmoid = SigmoidNode()

    def forward(self, newInput=None):
        # ability to update input value if input is from hidden layer
        if newInput:
            for i in range(len(newInput)):
                self.x[i].value = newInput[i]

        # do the multiplications
        multiplications = []
        for i in range(len(self.w)):
            multiplications.append(self.mul[i].forward(self.w[i], self.x[i]))

        # do the additions
        addition_result = multiplications[0]
        for i in range(len(multiplications) - 1): # there is n-1 add operators
            addition_result = self.add[i].forward(addition_result, multiplications[i + 1])

        # return the final sigmoid result
        return self.sigmoid.forward(addition_result)

    def backward(self):
        # back-propogate the errors
        self.sigmoid.backward()
        [op.backward() for op in self.add]
        [op.backward() for op in self.mul]

    def update(self):
        for w in self.w:
            w.value -= self.alpha * w.gradient

## Nerual Network Class

In [3]:
class NeuralNetwork(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha

        self.p1 = Perceptron([x1, x2], alpha=alpha)
        self.p2 = Perceptron([self.p1.forward().value], alpha=alpha)

    def forward(self):
        return self.p2.forward([self.p1.forward().value])

    def backward(self):
        self.p2.sigmoid.output.gradient = -2 * (target - self.p2.sigmoid.output.value)
        self.p2.backward()
        self.p1.sigmoid.output.gradient = self.p2.w[0].gradient  # propagate back the the weight from 2nd perceptron
        self.p1.backward()

    def update(self):
        self.p2.update()
        self.p1.update()

## Validation of 2 Perceptron Network

In [4]:
nn = NeuralNetwork(0.11, -1.0, alpha=0.1)
# number of iterations
N = 10000
# expected output
target = 0.3481972639817

nn.forward()
print('pass 1, value: %s' % nn.p2.sigmoid.output.value)
print('pass 1, hidden-value: %s' % nn.p1.sigmoid.output.value)

for i in range(N):
    # Step 1. Forward Pass
    nn.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t
    # square loss function.
    # nn.sigmoid.output.gradient = -2 * (target - nn.sigmoid.output.value)
    # Step 3. Backward Pass
    nn.backward()
    # Step 4. Update Weights and Bias
    nn.update()

print('pass %d, value: %s' % (N, nn.p2.sigmoid.output.value))
print('pass %d, hidden-value: %s' % (N, nn.p1.sigmoid.output.value))
print('weights:')
print('wx1_p1: %s' % nn.p1.w[0].value)
print('wx2_p1: %s' % nn.p1.w[1].value)
print('w-bias_p1: %s' % nn.p1.w[2].value)
print('wp1_p2: %s' % nn.p2.w[0].value)
print('w-bias_p2: %s' % nn.p2.w[1].value)

pass 1, value: 0.5271674916938318
pass 1, hidden-value: 0.5974372945096734
pass 10000, value: 0.3481972639817006
pass 10000, hidden-value: 0.5784757495611493
weights:
wx1_p1: 0.608744457018
wx2_p1: 0.539946966537
w-bias_p1: 0.789504393056738
wp1_p2: -0.36614788694707817
w-bias_p2: -0.4151651209798792


## Discussion
- Most of the learning seems to be happening in the second perceptron
- The output value of the first percepton doesn't change as much